<a href="https://colab.research.google.com/github/gemepolimi/gemepolimi/blob/main/NOAA_1889_1941.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# 1. This line opens the file you saved
# Make sure the file name matches exactly
try:
    df = pd.read_csv('noaa_data 2.txt', names=['Year', 'Month', 'Day', 'RawRate'])
except FileNotFoundError:
    print("Error: Could not find 'noaa_data 2.txt'. Make sure it is in the same folder.")
    exit()

# 2. CONVERSION: Multiply by 86,400 to get total mm for the day
df['Rain_mm'] = (df['RawRate'] * 86400).round(2) # Rounding to 2 decimal places like your image

# 3. Generate the Excel file with yearly sheets
with pd.ExcelWriter('Addis_Ababa_Daily_Rainfall 2.xlsx', engine='openpyxl') as writer:
    month_labels = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

    for year in range(df['Year'].min(), df['Year'].max() + 1):
        year_data = df[df['Year'] == year]

        if not year_data.empty:
            # Pivot into Day (rows) and Month (columns)
            table = year_data.pivot(index='Day', columns='Month', values='Rain_mm')

            # Ensure 31 days and 12 months are present, fill missing with 0.00
            table = table.reindex(index=range(1, 32), columns=range(1, 13), fill_value=0.0)

            # Rename columns to match your screenshot
            table.columns = month_labels

            # Add the 'Total Annual' calculation for each day
            table['Total Annual'] = table.sum(axis=1)

            # Write to a sheet named after the year
            table.to_excel(writer, sheet_name=str(year))

print("Success! Your Excel file 'Addis_Ababa_Daily_Rainfall 2.xlsx' is ready.")

Success! Your Excel file 'Addis_Ababa_Daily_Rainfall 2.xlsx' is ready.


In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment

def process_and_style_noaa(input_file, output_name):
    # 1. Load the data from your provided source files
    try:
        df = pd.read_csv(input_file, names=['Year', 'Month', 'Day', 'RawRate'], skipinitialspace=True)
    except FileNotFoundError:
        print(f"Error: Could not find {input_file}")
        return

    # 2. Conversion: kg/m^2/s to mm/day (86400 seconds in a day)
    df['Rain_mm'] = (df['RawRate'] * 86400).round(2)

    # 3. Create the Excel structure
    with pd.ExcelWriter(output_name, engine='openpyxl') as writer:
        month_labels = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

        for year in sorted(df['Year'].unique()):
            year_data = df[df['Year'] == year]

            # Pivot table: Day as rows, Month as columns
            table = year_data.pivot(index='Day', columns='Month', values='Rain_mm')
            table = table.reindex(index=range(1, 32), columns=range(1, 13), fill_value=0.0)
            table.columns = month_labels

            # Add Total Annual column and Monthly Total row
            table['Total Annual'] = table.sum(axis=1)
            table.loc['Monthly Total'] = table.sum(axis=0)

            table.to_excel(writer, sheet_name=str(year))

    # 4. Apply exactly requested styling
    wb = load_workbook(output_name)

    # Colors defined in your request
    fill_green = PatternFill(start_color="E2EFDA", end_color="E2EFDA", fill_type="solid")
    fill_blue = PatternFill(start_color="DDEBF7", end_color="DDEBF7", fill_type="solid")
    font_bold = Font(bold=True)
    center_align = Alignment(horizontal='center')

    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]

        # Style Header Row (Day, J, F... Total Annual)
        for col in range(1, 15):
            ws.cell(row=1, column=col).font = font_bold
            ws.cell(row=1, column=col).alignment = center_align

        # Style First Column (Day numbers) - Light Green
        for row in range(2, 33): # Days 1 to 31
            cell = ws.cell(row=row, column=1)
            cell.fill = fill_green
            cell.font = font_bold

        # Style Last Column (Total Annual) - Light Blue
        for row in range(2, 34): # Including the corner total
            cell = ws.cell(row=row, column=14)
            cell.fill = fill_blue
            cell.font = font_bold

        # Style Last Row (Monthly Total) - Light Blue
        for col in range(2, 14): # Months J through D
            cell = ws.cell(row=33, column=col)
            cell.fill = fill_blue
            cell.font = font_bold

    wb.save(output_name)
    print(f"File created and styled: {output_name}")

# Run the process for both your provided files
process_and_style_noaa('noaa_data.txt', 'Addis_Ababa_Rainfall_1889_1920.xlsx')
process_and_style_noaa('noaa_data 2.txt', 'Addis_Ababa_Rainfall_1921_1941.xlsx')

File created and styled: Addis_Ababa_Rainfall_1889_1920.xlsx
File created and styled: Addis_Ababa_Rainfall_1921_1941.xlsx
